In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import torch
from peft import PeftModel, PeftConfig

In [3]:
# 사용할 모델명 또는 경로 설정
model_name = 'beomi/llama-2-ko-7b'
peft_model_path = '/home/smkim/PatentClassification/llama-2-ko-7b-ft-paper_classification_ct/checkpoint-1406'

In [4]:
# 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)#.to('cuda')
# model.load_state_dict(torch.load(checkpoint_path))
model = PeftModel.from_pretrained(model, peft_model_path).to('cuda')

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [6]:
# model = model.merge_and_unload()
# model = model.to('cuda')

In [5]:
# 사용할 instruction 입력
# instruction = """Analyze this document and provide the corresponding technical field code according to the CPC classification."""
# instruction = """Analyze the content of this document and identify the relevant academic field according to the WoS category."""
instruction = """Examine the content of this document and identify the key Citation Topics it addresses."""

In [6]:
# input context
input_t= """Background A variety of minimally invasive surgical approaches are available as an alternative to transurethral resection of the prostate (TURP) for management of lower urinary tract symptoms (LUTS) in men with benign prostatic hyperplasia (BPH). Prostatic arterial embolization (PAE) is a relatively new, minimally invasive treatment approach. Objectives To assess the effects of PAE compared to other procedures for treatment of LUTS in men with BPH. Search methods We performed a comprehensive search the Cochrane Library, MEDLINE, Embase, three other databases, trials registries, other sources of grey literature, and conference proceedings with no restrictions on language of publication or publication status, up to 8 November 2021. Selection criteria We included parallel-group randomized controlled trials (RCTs), as well as non-randomized studies (NRS, limited to prospective cohort studies with concurrent comparison groups) enrolling men over the age of 40 years with LUTS attributed to BPH undergoing PAE versus TURP or other surgical interventions. Data collection and analysis Two review authors independently classified studies for inclusion or exclusion and abstracted data from the included studies. We performed statistical analyses by using a random-effects model and interpreted them according to the Cochrane Handbook for Systematic Reviews of Interventions. We used GRADE guidance to rate the certainty of evidence of RCTs and NRSs. Main results We found data to inform two comparisons: PAE versus TURP (six RCTs and two NRSs), and PAE versus sham (one RCT). Mean age was 66 years, International Prostate Symptom Score (IPSS) was 22.8, and prostate volume of participants was 72.8 mL. This abstract focuses on the comparison of PAE versus TURP as the primary topic of interest. Prostatic arterial embolization versus transurethral resection of the prostate We included six RCTs and two NRSs with short-term (up to 12 months) follow-up, and two RCTs and one NRS with long-term follow-up (13 to 24 months). Short-term follow-up: based on RCT evidence, there may be little to no difference in urologic symptom score improvement measured by the International Prostatic Symptom Score (IPSS) on a scale from 0 to 35, with higher scores indicating worse symptoms (mean difference [MD] 1.72, 95% confidence interval [CI] -0.37 to 3.81; 6 RCTs, 360 participants; I-2 = 78%; low-certainty evidence). There may be little to no difference in quality of life as measured by the IPSS-quality of life question on a scale from 0 to 6, with higher scores indicating worse quality of life between PAE and TURP, respectively (MD 0.28, 95% CI -0.28 to 0.84; 5 RCTs, 300 participants; I-2 = 63%; low-certainty evidence). While we are very uncertain about the effects of PAE on major adverse events (risk ratio [RR] 0.75, 95% Cl 0.19 to 2.97; 4 RCTs, 250 participants; I-2 = 24%; very low-certainty evidence), PAE likely increases retreatments (RR 3.20, 95% CI 1.41 to 7.27; 4 RCTs, 303 participants; I-2 = 0%; moderate-certainty evidence). PAE may make little to no difference in erectile function measured by the International Index of Erectile Function-5 on a scale from 1 to 25, with higher scores indicating better function (MD -0.50 points, 95% CI -5.88 to 4.88; 2 RCTs, 120 participants; I-2 = 68%; low-certainty evidence). Based on NRS evidence, PAE may reduce the occurrence of ejaculatory disorders (RR 0.51, 95% CI 0.35 to 0.73; 1 NRS, 260 participants; low-certainty evidence). Long-term follow-up: based on RCT evidence, PAE may result in little to no difference in urologic symptom scores (MD 2.58 points, 95% CI - 1.54 to 6.71; 2 RCTs, 176 participants; I-2 = 73%; low-certainty evidence) and quality of life (MD 0.50 points, 95% CI -0.03 to 1.04; 2 RCTs, 176 participants; I-2 = 29%; low-certainty evidence). We are very uncertain about major adverse events (RR 0.91, 95% CI 0.20 to 4.05; 2 RCTs, 206 participants; I-2 = 72%; very low-certainty evidence). PAE likely increases retreatments (RR 3.80, 95% CI 1.32 to 10.93; 1 RCT, 81 participants; moderate-certainty evidence). While PAE may result in little to no difference in erectile function (MD 3.09 points, 95% CI -0.76 to 6.94; 1 RCT, 81 participants; low-certainty evidence), PAE may reduce the occurrence of ejaculatory disorders (RR 0.67, 95% CI 0.45 to 0.98; 1 RCT, 50 participants; low-certainty evidence). Authors' conclusions Compared to TURP, PAE may provide similar improvement in urologic symptom scores and quality of life. While we are very uncertain about major adverse events, PAE likely increases retreatment rates. While erectile function may be similar, PAE may reduce ejaculatory disorders. Certainty of evidence for the outcomes of this review was low or very low except for retreatment (moderate-certainty evidence), signaling that ou"""

In [7]:
# 프롬프트 포맷
input_text = f"""[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

{input_t}

{instruction} [/INST]

"""

In [8]:
# 생성 및 출력
input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=4096, truncation=True).to('cuda')
# outputs = model.generate(input_ids, max_new_tokens=100)
outputs = model.generate(input_ids, max_new_tokens=100, do_sample=True, top_k=40, top_p=0.92, no_repeat_ngram_size=5,)
decoded_outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_outputs)

[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

Background A variety of minimally invasive surgical approaches are available as an alternative to transurethral resection of the prostate (TURP) for management of lower urinary tract symptoms (LUTS) in men with benign prostatic hyperplasia (BPH). Prostatic arterial embolization (PAE) is a relatively new, minimally invasive treatment approach. Objectives To assess the effects of PAE compared to other procedures for treatment of LUTS in men with BPH. Search methods We performed a comprehensive search the Cochrane Library, MEDLINE, Embase, three other databases, trials registries, other sources of grey literature, and conference proceedings with no restrictions on language of publication or publication status, up to 8 November 2021. Selection criteria We included parallel-group randomized controlled trials (RCTs), as well as non-randomized studies (NRS, limited to prospective cohort studies with concurrent comparison groups) enrolling 